In [439]:
import pandas as pd

customers = pd.read_csv("1customers_raw.csv")
products = pd.read_csv("products_raw.csv")
sales = pd.read_csv("sales_raw.csv")

print(customers.shape)
print(products.shape)
print(sales.shape)


(26, 7)
(20, 5)
(41, 7)


In [440]:
customers_before = len(customers)
customers = customers.drop_duplicates()
customers_after = len(customers)

products = products.drop_duplicates()
sales = sales.drop_duplicates(subset=["transaction_id"])


In [441]:
customers["email"] = customers["email"].fillna(
    "unknown_" + customers["customer_id"] + "@example.com"
)


In [442]:
import re
import pandas as pd

def clean_phone(phone):
    if pd.isna(phone):
        return None
    # Convert phone to string to ensure re.sub receives a string
    phone_str = str(phone)
    digits = re.sub(r"\D", "", phone_str)
    if digits.startswith("91"):
        digits = digits[2:]
    if digits.startswith("0"):
        digits = digits[1:]
    return "+91-" + digits if len(digits) == 10 else None

customers["phone"] = customers["phone"].apply(clean_phone)

In [443]:
from datetime import datetime

def fix_date(date):
    if pd.isna(date):
        return None
    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m-%d-%Y", "%m/%d/%Y"):
        try:
            return datetime.strptime(date, fmt).strftime("%Y-%m-%d")
        except:
            continue
    return None

customers["registration_date"] = customers["registration_date"].apply(fix_date)
sales["transaction_date"] = sales["transaction_date"].apply(fix_date)


In [444]:
products["category"] = products["category"].str.strip().str.capitalize()


In [445]:
products["price"] = products["price"].fillna(products["price"].median())
products["stock_quantity"] = products["stock_quantity"].fillna(0)


In [446]:
sales = sales.dropna(subset=["customer_id", "product_id"])


In [447]:
import sqlite3

conn = sqlite3.connect("fleximart LAST.db")
cursor = conn.cursor()


In [448]:
customers.to_sql("customers", conn, if_exists="append", index=False)
products.to_sql("products", conn, if_exists="append", index=False)


20

In [449]:
with open("data_quality_report.txt", "w") as f:
    f.write(f"Customers processed: {customers_before}\n")
    f.write(f"Customers loaded: {len(customers)}\n")
    f.write("Duplicates removed and missing values handled\n")


In [450]:
with open("data_quality_report.txt", "w") as f:
    f.write(f"Customers processed: {customers_before}\n")
    f.write(f"Customers loaded: {len(customers)}\n")
    sales_before = len(sales)
    f.write(f"Sales processed: {sales_before}\n")
    f.write(f"Sales loaded: {len(sales)}\n")
    f.write("Duplicates removed and missing values handled successfully\n")

In [451]:
import pandas as pd

sales = pd.read_csv("sales_raw.csv") # This line is now uncommented to ensure sales is defined

# Drop invalid rows
sales = sales.dropna(subset=["customer_id", "product_id"])

# Fix date
sales["transaction_date"] = sales["transaction_date"].apply(fix_date)

sales.head()

,transaction_id,customer_id,product_id,quantity,unit_price,transaction_date,status
0,T001,C001,P001,1,45999,2024-01-15,Completed
1,T002,C002,P004,2,2999,2024-01-16,Completed
2,T003,C003,P007,1,52999,2024-01-15,Completed
4,T005,C005,P009,3,650,2024-01-20,Completed
5,T006,C006,P012,1,12999,2024-01-22,Completed


In [452]:
sales = pd.read_csv("sales_raw.csv")

# Drop invalid rows
sales = sales.dropna(subset=["customer_id", "product_id"])

# Fix date
sales["transaction_date"] = sales["transaction_date"].apply(fix_date)

sales.head()


,transaction_id,customer_id,product_id,quantity,unit_price,transaction_date,status
0,T001,C001,P001,1,45999,2024-01-15,Completed
1,T002,C002,P004,2,2999,2024-01-16,Completed
2,T003,C003,P007,1,52999,2024-01-15,Completed
4,T005,C005,P009,3,650,2024-01-20,Completed
5,T006,C006,P012,1,12999,2024-01-22,Completed


In [453]:
import sqlite3

conn = sqlite3.connect("fleximart LAST.db")

order_map = pd.read_sql(
    "SELECT customer_id, order_date FROM orders",
    conn
)


In [454]:
order_items_clean = sales.merge(
    order_map,
    left_on=["customer_id", "transaction_date"],
    right_on=["customer_id", "order_date"],
    how="left"
)

order_items_clean["subtotal"] = (
    order_items_clean["quantity"] * order_items_clean["unit_price"]
)


In [455]:
order_items_clean[[
    "transaction_id",
    "product_id",
    "quantity",
    "unit_price",
    "subtotal"
]].to_sql(
    "order_items",
    conn,
    if_exists="append",
    index=False
)

conn.close()

In [456]:
from google.colab import files
files.download("fleximart LAST.db")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>